Geophysics:  
- 3.33 inches for one-column figures
- 4.33 inches, for one-and-one-third-column

- graph labels 8-point sans serif font such as Arial or Helvetica?
- first letter of graph labels capitalized
- abscissa and ordinate of each graph labeled, units denoted in parentheses
- Title heading for each graph
- en dash instead of a hyphen to denote subtraction and negative numbers?
- Consistent style with those in other figures, especially similar figures?
- labels on vertical axes: read from left to right when you rotate the page **clockwise 90°**), centered vertically?
- Are scalars italicized consistently in text, figures, and figure captions?


In [ ]:
# Set rc params
# https://matplotlib.org/stable/tutorials/introductory/customizing.html#the-matplotlibrc-file

import matplotlib as mpl

mpl.rcParams["font.size"] = 8.0
mpl.rcParams["figure.dpi"] = 100 #600
mpl.rcParams["figure.figsize"] = (4.3, 8.6)
mpl.rcParams["figure.constrained_layout.use"] = True
mpl.rcParams["image.origin"] = "lower"


In [ ]:
# Imports

%load_ext autoreload
%autoreload 3

from functools import partial
from pathlib import Path

import colorcet as cc
import matplotlib.pyplot as plt
import numpy as np
import torch
import yaml
from PIL import Image
from torch.utils.data import DataLoader, Subset
from tqdm.auto import tqdm

import datasets as dsets
import models
import utils
from test import reshape, batched_predict
from mlnoddy.datasets import parse_geophysics, Norm

import rasterio
import tifffile
from datasets.noddyverse import HRLRNoddyverse, NoddyverseWrapper
from datasets.noddyverse import load_naprstek_synthetic as load_naprstek

from inference import (
    CustomTestDataset,
    load_model,
    load_real_data,
    input_tiler,
    real_inference,
    test_custom_data,
    eval,
    plot_gt,
    plot_canny,
    plt_results,
    save_pred,
)

with open("configs/train_swinir-lte_geo.yaml", "r") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
# Calculate Line spacings, sample factors, cell sizes, etc #


def tabulate_scale_info(
    hr_scale_fac=4,  # Every nth line for HR
    cs_factor=4,  # 1/4 per Reid 1980, 1/5 per GA
    crop_dim=180,  # cells
    max_scale=10,
):
    """Print Line spacings, sample factors, cell sizes, etc, per scale"""
    cs_factor = 1 / cs_factor
    noddy_cs = 20  # m
    noddy_dim = 200  # cells
    noddy_extent = noddy_cs * noddy_dim  # m
    crop_extent = crop_dim * noddy_cs
    scale_factors = range(1, max_scale + 1)

    hr_line_spacing = noddy_cs * hr_scale_fac
    print("Scale | Line spacing | Cell size | LR Dimensions | Sample Q")
    for scale in scale_factors:
        print(
            f" {scale:02} x |"
            f"     {(ls := hr_scale_fac * scale * noddy_cs):03} m    |"
            f"   {(cs := ls * cs_factor):03.0f} m   |"
            f"  {(dim := crop_extent / cs):05.1f} cells  |"
            f" {(dim**2):03.1f}"
        )


def dumpforcfg(hr_scale_fac=4, cs_factor=4, crop_dim=180, max_scale=10):
    """To copy and paste values into cfg"""

    cs_factor = 1 / cs_factor
    noddy_cs = 20  # m
    noddy_dim = 200  # cells
    noddy_extent = noddy_cs * noddy_dim  # m
    crop_extent = crop_dim * noddy_cs
    scale_factors = range(1, max_scale + 1)

    hr_line_spacing = noddy_cs * hr_scale_fac

    for scale in scale_factors:
        ls = hr_scale_fac * scale * noddy_cs
        cs = ls * cs_factor

        print(
            "\n"
            f"      inp_size: {(dim := crop_extent / cs):0.0f}\n"
            f"      sample_q: {(dim**2):03.0f}\n"
            f"      scale_min: {scale}\n"
            f"      scale_max: {scale}",
        )


# tabulate_scale_info(hr_scale_fac=4)
dumpforcfg(hr_scale_fac=4)


In [ ]:
## Plot Noddy model with specific ID ##

with open("configs/train_swinir-lte_geo.yaml", "r") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

args = cfg["train_dataset"]["dataset"]["args"]
dset = HRLRNoddyverse(
    root_path=args["root_path"],
    norm=args["norm"],
    noddylist=args["noddylist"],
    blocklist=args["blocklist"],
    hr_line_spacing=args["hr_line_spacing"],
    load_magnetics=args["load_magnetics"],
)
dset.scale = 4
unorm = Norm(args["norm"][0], args["norm"][1]).inverse_mmc

noddydir = "C:/Users/Public/scratch/Noddy_1M/PLUG_PLUG_FOLD/models_by_code/models/PLUG_PLUG_FOLD/20-09-10-14-45-47-034629522"
noddystr = "20-09-10-14-45-47-034629522"

for sample_id in (
    -7955,
    -7904,
    -7844,
    -7826,
    -7779,
    -7700
    # range(
    # cfg["val_dataset"]["dataset"]["args"]["use_dset_slice"][0],
    # cfg["val_dataset"]["dataset"]["args"]["use_dset_slice"][1])
):
    plt.figure()
    plt.title(f"Sample {sample_id}")
    plt.imshow(dset[sample_id]["gt_grid"].squeeze())
    plt.show()


In [ ]:
## Plot all GT and low resolution grids ##

with open("configs/train_swinir-lte_geo.yaml", "r") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

sample_id = cfg["val_dataset"]["dataset"]["args"]["use_dset_slice"][0]
sample_id += 29
sample_id = -7779
max_scale = 6
scale_range = (6,7)  # range(1, max_scale + 1, 2)


args = cfg["train_dataset"]["dataset"]["args"]
dset = HRLRNoddyverse(
    root_path=args["root_path"],
    norm=args["norm"],
    noddylist=args["noddylist"],
    blocklist=args["blocklist"],
    hr_line_spacing=args["hr_line_spacing"],
    load_magnetics=args["load_magnetics"],
)
dset.scale = 1
unorm = Norm(args["norm"][0], args["norm"][1]).inverse_mmc

grids = {}

for scale in tqdm(scale_range):
    dset.scale = scale
    grids[f"{scale}x"] = (dset[sample_id]["gt_grid"], dset[sample_id]["lr_grid"])


fig, axes = plt.subplots(2, len(scale_range))
plt.suptitle(f"Sample {sample_id}")
for [axgt, axlr], grid in zip(axes.T, grids):
    axgt.set_title(grid)
    plot_gt(axgt, grids[grid][0].squeeze(), args, scale=int(grid[:-1]))

    cbar = axlr.imshow(
        unorm(grids[grid][1]).squeeze(),
        interpolation="nearest",
        origin="lower",
        cmap=cc.cm.CET_L1,
    )
    plt.colorbar(mappable=cbar, ax=axlr, orientation="horizontal")

plt.savefig(f"LR samples_{sample_id}.png", facecolor="white", transparent=False)
# plt.show()


In [ ]:
### TEST IQA ###

from pathlib import Path
import piq
import torch

import datasets
from mlnoddy.datasets import Norm, parse_geophysics

norm = Norm(-10000, 10000, out_vals=(0, 1)).min_max_clip
unnorm = Norm(-10000, 10000, out_vals=(0, 1)).inverse_mmc

t1 = parse_geophysics(Path("D:/luke/Noddy_data/Noddy_1M/DYKE_DYKE_DYKE/models_by_code/models/DYKE_DYKE_DYKE/20-09-04-15-13-14-866983543.his"), mag=True)
t1 = torch.from_numpy(next(t1)).unsqueeze(0).unsqueeze(0)
t1 = norm(t1)

t1.clamp_(0,1)
# print(t1)

t2 = parse_geophysics(Path("D:/luke/Noddy_data/Noddy_1M/DYKE_DYKE_DYKE/models_by_code/models/DYKE_DYKE_DYKE/20-09-04-15-13-14-866983543.his"), mag=True)
t2 = torch.from_numpy(next(t2)).unsqueeze(0).unsqueeze(0)
t2 = norm(t2) - torch.rand_like(t2) * 0.1
t2.clamp_(0,1)
# print(t2)

print(piq.FSIMLoss(chromatic=False)(t1, t2))

import matplotlib.pyplot as plt

plt.figure()
plt.imshow(t1.numpy().squeeze())
plt.figure()
plt.imshow(t2.numpy().squeeze())


In [ ]:
### Plot performance of all models on Noddy Synthetic ###

with open("configs/inference.yaml", "r") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

names = cfg["range_model_names"][::-1][:1]
scales = cfg["range_model_scales"][::-1][:1]

# inp_sizes = 180 / scales


# Define Data ###
spec = cfg["test_dataset"]
dataset = dsets.make(spec["dataset"])
dataset = dsets.make(spec["wrapper"], args={"dataset": dataset})

if cfg["limit_to_plots"]:
    samples = cfg["plot_samples"][1]
    samples.extend(cfg["plot_samples"][0])
    dataset = Subset(dataset, samples)

loader = DataLoader(
    dataset,
    batch_size=spec["batch_size"],
    num_workers=cfg.get("num_workers"),
    persistent_workers=bool(cfg.get("num_workers")),
    pin_memory=True,
)

In [ ]:
for model_name, scale in tqdm(zip(names, scales), total=len(names)):
    print(model_name, scale)
    ### Pack Options
    model_dir = Path(cfg["model_dir"])
    model_paths = list(model_dir.glob(f"**/*{model_name}*last.pth"))
    if len(model_paths) != 1:
        raise FileNotFoundError()
    model_spec = torch.load(model_paths[0], map_location="cuda")["model"]
    model = models.make(model_spec, load_sd=True).to("cuda")
    opts = dict(
        model_name=model_name,
        model_path=model_paths[0],
        save_path=Path(cfg["inference_output_dir"] or f"inference/{model_name}"),
        rgb_range=cfg["rgb_range"],
        shave_factor=0,  # pixels to shave (edges may include NaN)
        ids=cfg["plot_samples"],  # Sample IDs
        mag=cfg["test_dataset"]["dataset"]["args"]["load_magnetics"],
        grv=cfg["test_dataset"]["dataset"]["args"]["load_gravity"],
        eval_bsize=cfg["eval_bsize"],
        limit_to_plots=cfg["limit_to_plots"],
        gt_list=cfg["test_dataset"]["dataset"]["args"]["root_path"],
    )

    scale_min = spec["wrapper"]["args"]["scale_min"]
    scale_max = spec["wrapper"]["args"]["scale_max"]

    print(
        f"\nModel: {opts['model_path'].absolute()}\n"
        f"Saving to: {opts['save_path'].absolute()}"
    )


In [ ]:
### Plot real data via HRLR dataset ###

import argparse
import os
import random
from datetime import datetime
from pathlib import Path
import yaml

from comet_ml import Experiment
import numpy as np
import piq
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torch.optim.lr_scheduler import MultiStepLR
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm

import datasets
import models
import utils
from test import reshape, eval_psnr

import matplotlib.pyplot as plt

index = 10

with open(r"D:\luke\lte_geo\configs\train_swinir-lte_geo.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

spec = config.get("train_dataset")

dataset = datasets.make(spec["dataset"])
dataset = datasets.make(spec["wrapper"], args={"dataset": dataset})

dataset.dataset.scale = 6

lr = dataset.dataset[index]["lr_grid"]
gt = dataset.dataset[index]["hr_grid"]

plt.figure()
plt.imshow(lr.squeeze())
plt.figure()
plt.imshow(gt.squeeze())



In [ ]:
## Do Inference ###
# results_dict = {}
# custom_results_dict = {}

# scale_min = 4
# scale_max = 4

# pbar_m = tqdm(range(scale_min, scale_max + 1))
# for scale in pbar_m:
#     pbar_m.set_description(f"{scale}x scale")
#     opts["shave"] = scale * opts["shave_factor"]

#     # lazy way to reset custom grid opts
#     opts["ids"] = cfg["plot_samples"]
#     opts["gt"] = None
#     opts["set"] = "test"

#     dataset.scale = scale
#     dataset.scale_min = scale
#     dataset.scale_max = scale
#     if cfg["limit_to_plots"]:
#         # Not sure how to better handle Subset dataset
#         dataset.dataset.scale = scale
#         dataset.dataset.scale_min = scale
#         dataset.dataset.scale_max = scale

#     results = eval(model, scale, loader, opts, cfg=cfg)
#     results_dict[f"{scale}x"] = results
#     pbar_m.write(f"{scale}x scale - Mean:")
#     pbar_m.write(
#         ", ".join(
#             f"{metric_name}: {metric_value:.4f}"
#             for metric_name, metric_value in results.items()
#         )
#     )


In [ ]:
# plt_results(results_dict, opts)


In [ ]:
model.eval()
for i, batch in enumerate(loader):
    inp = batch["inp"].to("cuda", non_blocking=True)
    coord = batch["coord"].to("cuda", non_blocking=True)
    cell = batch["cell"].to("cuda", non_blocking=True)
    # batch["gt"] = batch["gt"].to("cuda", non_blocking=True)

    with torch.no_grad():
        if opts["eval_bsize"]:
            pred = batched_predict(model, inp, coord, cell, opts["eval_bsize"])
        else:
            pred = model(inp, coord, cell)

    pred, batch = reshape(batch, h_pad=0, w_pad=0, coord=coord, pred=pred)
    lr = batch["inp"].detach().cpu().squeeze().numpy()
    hr = batch["gt"].detach().cpu().squeeze().numpy()
    sr = pred.detach().cpu().squeeze().numpy()
    break


In [ ]:
# import cv2
from inference import plot_canny

fig, [ax0, ax1] = plt.subplots(1, 2)

c0 = ax0.imshow(sr, cmap=cc.cm.CET_L8, origin="lower", interpolation="nearest")
plt.colorbar(c0, orientation="horizontal", ax=ax0)
plot_canny(
    ax1,
    hr,
    sr,
    np.array(Image.fromarray(lr).resize(hr.shape, Image.Resampling.BICUBIC)),
    sigma=1,
)


In [ ]:
mask = np.zeros_like(hr)
mask[:, ::4] = 1

# cmap.set_bad(alpha=0)
fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(10, 10), dpi=600)
cgry = ax.imshow(hr, cmap=cc.cm.CET_L1, interpolation="nearest", origin="lower")
cclr = ax.imshow(
    hr, cmap=cc.cm.CET_L8, interpolation="nearest", origin="lower", alpha=mask
)
plt.colorbar(mappable=cgry, ax=ax, orientation="horizontal", label="Unsampled")
plt.colorbar(mappable=cclr, ax=ax, orientation="horizontal", label="Line Sampled")
